In [10]:
from IPython.core.display import display, HTML, Markdown
from ipywidgets import Button, HBox, VBox, widgets, Layout
from tabulate import tabulate
import pandas as pd

In [11]:
#solver solo per non fare i calcoli a mano

def num_percorsi(env,r,c):
    m = []
    for i in range(0,r):
        m.append([0 for j in range(0,c)])
    hole = 0
    i=0
    j=0
    for i in range(0,r):
        m[i][0]=0
    for j in range(0,c):
        m[0][j]=0
    for i in range(0,r):
        for j in range(0,c):
            if i==1 and j==1:
                m[i][j]=1
            else:
                if str(env[i][j])=="X":
                    m[i][j]="X"
                else:
                    if m[i-1][j] == "X" and m[i][j-1] == "X":
                         m[i][j] =0                    
                    elif m[i-1][j] == "X":
                        m[i][j]= m[i][j-1];
                    elif m[i][j-1] == "X":
                        m[i][j]= m[i-1][j]
                    else:
                        m[i][j] = m[i-1][j] + m[i][j-1];
        
    return m;


            


In [45]:
def evaluation_format(answ, pt_green,pt_red):
    if pt_green!=0:
        pt_blue=pt_red-pt_green
        pt_red=0
    return f"{answ}. Totalizzeresti <span style='color:green'>[{pt_green} safe pt]</span>, \
                                    <span style='color:blue'>[{pt_blue} possible pt]</span>, \
                                    <span style='color:red'>[{pt_red} out of reach pt]</span>.<br>"

def check_num_paths_to_goal(num_path, row, col):
    for i in range (0,row-1):
        if num_path[i][0]!=0:
            return evaluation_format("No", 0, 10)+"Attenzione, il numero di cammini parte dalla cella $(1,1)$"
    for j in range (0,col-1):
        if num_path[0][j]!=0:
            return evaluation_format("No", 0, 10)+"Attenzione, il numero di cammini parte dalla cella $(1,1)$"
    for i in range(row-1,0,-1):
        for j in range (col-1,0,-1):
            #print(f"{i},{j} -> {num_path[i][j]}={num_path[i-1][j]}+{num_path[i][j-1]}")
            if num_path[i][j]!="X":
                if i==1 and j==1:
                    if num_path[i][j]!=1:
                        return  evaluation_format("No", 0, 10)+"Il valore nella cella $(1,1)$ non è corretto"
                elif num_path[i-1][j]!="X" and num_path[i][j-1]!="X":
                    if num_path[i][j]!=num_path[i-1][j]+num_path[i][j-1]:
                        return  evaluation_format("No", 0, 10)+"La tua soluzione non è coerente"
                elif num_path[i-1][j]!="X":
                    if num_path[i][j]!=num_path[i-1][j]:
                        return  evaluation_format("No", 0, 10)+"La tua soluzione non è coerente"
                elif num_path[i][j-1]!="X":
                    if num_path[i][j]!=num_path[i][j-1]:
                        return  evaluation_format("No", 0, 10)+"La tua soluzione non è coerente"
                else:
                    if num_path[i][j]!=0:
                        return  evaluation_format("No", 0, 10)+"La tua soluzione non è coerente"
    return  evaluation_format("Si", 1, 10)+"La tua soluzione è coerente"
        
                
                        
            

## Esercizio \[60 pts\]
(campo minato) Ricerca dei cammini.

Un robot R, inizialmente situato nella cella $A1$, deve portarsi nella sua home H situata nella cella $H9$.

In [13]:
table = [
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, "R" , 2 , 3 , 1 , 1 , 1 , 0 , 0 , "X" ],
         [0, 3 , 3 , 1 , 0 , "X" , "X" , 0 , 0 , 0 ],
         [0, 2 , "X" , 0 , "X" , 0 , 0 , 1 , 1 , 1 ],
         [0, 0 , 0 , 1 , 0 , 0 , 0 , 1 , "X" , 0 ],
         [0, 0 , 0 , "X" , 1 , "X" , 1 , 0 , 0 , 0 ],
         [0, 0 , 1 , 1 , 1 , 0 , 3 , "X" , 0 , 1 ],
         [0, 3 , "X" , 0 , 1 , 2 , 0 , 0 , 1 , 0 ],
         [0, 2 , 1 , 2 , 1 , 2 , 1 , 2 , 1 , "H"] ,
        ]


columns=["1","2","3","4","5","6","7","8","9"]
index=pd.Index(["0","A","B","C","D","E","F","G","H"])
df=pd.DataFrame(table,index=index)
df = df.iloc[1:,1:]



print(tabulate(df, headers=columns, tablefmt='fancy_grid'))
#display(HTML(df.to_html()))

╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │ 1   │ 2   │ 3   │ 4   │ 5   │ 6   │ 7   │ 8   │ 9   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│ A  │ R   │ 2   │ 3   │ 1   │ 1   │ 1   │ 0   │ 0   │ X   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ B  │ 3   │ 3   │ 1   │ 0   │ X   │ X   │ 0   │ 0   │ 0   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ C  │ 2   │ X   │ 0   │ X   │ 0   │ 0   │ 1   │ 1   │ 1   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ D  │ 0   │ 0   │ 1   │ 0   │ 0   │ 0   │ 1   │ X   │ 0   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ E  │ 0   │ 0   │ X   │ 1   │ X   │ 1   │ 0   │ 0   │ 0   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ F  │ 0   │ 1   │ 1   │ 1   │ 0   │ 3   │ X   │ 0   │ 1   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ G  │ 3   │ X   │ 0   │ 1   │ 2   │ 0   │ 0   │ 1   │ 0   │
├────┼─────┼─────┼─────┼

Ogni cella senza mina contiene un numero di monetine come rappresentato.\
Quante monetine può raccogliere al massimo il robot?

I movimenti base possibili sono il passo verso destra (ad esempio dalla cella A4 alla cella A5) ed il passo verso in basso (ad esempio dalla cella A4 alla cella B4).\
Tuttavia il robot non può visitare le celle occupate da un Pacman (indicate con la X).

__Richieste__:

1. __\[10 pts\]__ Costrisci una matrice *num_paths_to_goal* di dimensione $(m+1)x(n+1)$ dove per ogni $i = 0,..., m$ e $j = 0,..., n$ si riporti il numero di cammini dalla cella $(1,1)$ alla cella $(i,j)$

In [14]:
num_paths_to_goal=[[],[],[],[],[],[],[],[],[]]
num_paths_to_goal= num_percorsi(table,len(table),len(table[0]))



In [15]:
print("La soluzione proposta è la seguente:")
df2=pd.DataFrame(num_paths_to_goal,index=index)
df2 = df2.iloc[1:,1:]
print(tabulate(df2, headers=columns, tablefmt='fancy_grid'))

La soluzione proposta è la seguente:
╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │   1 │ 2   │ 3   │ 4   │ 5   │ 6   │ 7   │ 8   │ 9   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│ A  │   1 │ 1   │ 1   │ 1   │ 1   │ 1   │ 1   │ 1   │ X   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ B  │   1 │ 2   │ 3   │ 4   │ X   │ X   │ 1   │ 2   │ 2   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ C  │   1 │ X   │ 3   │ X   │ 0   │ 0   │ 1   │ 3   │ 5   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ D  │   1 │ 1   │ 4   │ 4   │ 4   │ 4   │ 5   │ X   │ 5   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ E  │   1 │ 2   │ X   │ 4   │ X   │ 4   │ 9   │ 9   │ 14  │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ F  │   1 │ 3   │ 3   │ 7   │ 7   │ 11  │ X   │ 9   │ 23  │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ G  │   1 │ X   │ 3   │ 10  │ 17  │ 28  │ 28  │

In [46]:
display(Markdown(check_num_paths_to_goal(num_paths_to_goal,len(num_paths_to_goal),len(num_paths_to_goal[0]))))

8,9 -> 184=60+124
8,8 -> 124=37+87
8,7 -> 87=28+59
8,6 -> 59=28+31
8,5 -> 31=17+14
8,4 -> 14=10+4
8,3 -> 4=3+1
8,2 -> 1=X+1
8,1 -> 1=1+0
7,9 -> 60=23+37
7,8 -> 37=9+28
7,7 -> 28=X+28
7,6 -> 28=11+17
7,5 -> 17=7+10
7,4 -> 10=7+3
7,3 -> 3=3+X
7,2 -> X=3+1
7,1 -> 1=1+0
6,9 -> 23=14+9
6,8 -> 9=9+X
6,7 -> X=9+11
6,6 -> 11=4+7
6,5 -> 7=X+7
6,4 -> 7=4+3
6,3 -> 3=X+3
6,2 -> 3=2+1
6,1 -> 1=1+0
5,9 -> 14=5+9
5,8 -> 9=X+9
5,7 -> 9=5+4
5,6 -> 4=4+X
5,5 -> X=4+4
5,4 -> 4=4+X
5,3 -> X=4+2
5,2 -> 2=1+1
5,1 -> 1=1+0
4,9 -> 5=5+X
4,8 -> X=3+5
4,7 -> 5=1+4
4,6 -> 4=0+4
4,5 -> 4=0+4
4,4 -> 4=X+4
4,3 -> 4=3+1
4,2 -> 1=X+1
4,1 -> 1=1+0
3,9 -> 5=2+3
3,8 -> 3=2+1
3,7 -> 1=1+0
3,6 -> 0=X+0
3,5 -> 0=X+X
3,4 -> X=4+3
3,3 -> 3=3+X
3,2 -> X=2+1
3,1 -> 1=1+0
2,9 -> 2=X+2
2,8 -> 2=1+1
2,7 -> 1=1+X
2,6 -> X=1+X
2,5 -> X=1+4
2,4 -> 4=1+3
2,3 -> 3=1+2
2,2 -> 2=1+1
2,1 -> 1=1+0
1,9 -> X=0+1
1,8 -> 1=0+1
1,7 -> 1=0+1
1,6 -> 1=0+1
1,5 -> 1=0+1
1,4 -> 1=0+1
1,3 -> 1=0+1
1,2 -> 1=0+1
1,1 -> 1=0+0


Si. Totalizzeresti <span style='color:green'>[1 safe pt]</span>,                                     <span style='color:blue'>[9 possible pt]</span>,                                     <span style='color:red'>[0 out of reach pt]</span>.<br>La tua soluzione è coerente

1. __\[10 pts\]__ Quanti sono i percorsi possibili se la partenza è in A–1?

2. __\[10 pts\]__ E se la partenza è in B–3?

3. __\[10 pts\]__ E se con partenza in A–1 il robot deve giungere in F–6?

4. __\[10 pts\]__ E se con partenza in A–1 ed arrivo in H–9 al robot viene richiesto di passare per
la cella D–5?

5. __\[10 pts\]__ Quale é il massimo valore in premi raccoglibili lungo una traversata da A–1 a H–9?

6. __\[10 pts\]__ Quanti sono i percorsi possibili che assicurino di portare a case tale massimo valore?